In [37]:
%matplotlib ipympl
import os
import os.path as path
import pandas as pd
import h5py
import numpy as np
from faker import Factory
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import plotly
from ipywidgets import fixed, interact_manual, widgets
import plotly.graph_objects as go
import plotly.express as plot_x

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, data_tools, variables, calibration, data_loadcrop, ion_selection, reconstruction, selectors_data
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.calibration_tools import dataset_path_qt
from pyccapt.calibration.calibration_tools import hdf_to_leap

In [2]:
button = widgets.Button(
    description='load dataset',
)

@button.on_click
def open_file_on_click(b):
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [15]:
tdc, det_diam, pulse_mode, flightPathLength, t0_d, max_tof = wd.dataset_tdc_selection()
display(tdc, pulse_mode, flightPathLength)

Dropdown(description='TDC model:', options=('surface_concept', 'roentdec'), value='surface_concept')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

$$\textbf{You can specify which dataset to use in below block}$$

In [47]:
tdc_model = tdc.value
pulse_mode_ini = pulse_mode.value
flight_path_length = flightPathLength

dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
dataset_name = os.path.splitext(dataset_name_with_extention)[0]


variables.init()
# variables.path = os.path.join(p, 'tests//data')
variables.result_path = os.path.dirname(dataset_main_path) + '/reconstruction/'
if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
        
filename = dataset_path

head, tail = os.path.split(filename)
figname = os.path.splitext(tail)[0]

data = data_tools.read_hdf5_through_pandas(filename)
range_data = None

In [5]:
data

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse (V),start_counter,t (ns),t_c (nm),x_det (cm),y_det (cm),pulse_pi,ion_pp,tof_c (ns)
0,0.0,0.0,0.0,27.352441,27.401002,5269.834473,5269.834473,54436.0,635.421132,0.0,-1.595020,-1.661878,0,1,0.0
1,0.0,0.0,0.0,27.228966,27.522766,5269.834473,5269.834473,54903.0,635.901192,0.0,0.646286,2.129878,467,1,0.0
2,0.0,0.0,0.0,27.086795,27.582728,5269.834473,5269.834473,55353.0,648.101574,0.0,-0.175102,3.155020,450,1,0.0
3,0.0,0.0,0.0,27.282802,26.913047,5269.834473,5269.834473,55831.0,636.010920,0.0,1.964327,-1.999347,478,1,0.0
4,0.0,0.0,0.0,27.132434,27.273478,5269.834473,5269.834473,56022.0,633.350016,0.0,1.381714,1.757388,191,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898626,0.0,0.0,0.0,248.242374,248.363850,5711.471680,5711.471680,37587.0,1716.831720,0.0,0.369306,-1.330776,83,1,0.0
1898627,0.0,0.0,0.0,27.040206,27.320343,5711.471680,5711.471680,38162.0,600.993972,0.0,0.423429,0.630367,575,1,0.0
1898628,0.0,0.0,0.0,27.665819,27.881308,5711.471680,5711.471680,38200.0,606.096324,0.0,0.560327,0.200571,38,1,0.0
1898629,0.0,0.0,0.0,27.019084,27.072135,5711.471680,5711.471680,38467.0,600.026994,0.0,0.926449,-0.633551,267,1,0.0


In [53]:
# exctract needed data from Pandas data frame as an numpy array
dld_highVoltage = data['high_voltage (V)'].to_numpy()
dld_x = data['x_det (cm)'].to_numpy()
dld_y = data['y_det (cm)'].to_numpy()
mc = data['mc_c (Da)'].to_numpy()

In [54]:
def mc_hist_plot(mc, bin_size, prominence, distance, percent, selector, plot, figname, lim_mc):
    peaks_ini, peaks_y_ini, peak_widths_p_ini = tools.hist_plot(mc[mc < lim_mc], bin_size, distance=distance, percent=percent, prominence=prominence, selector=selector, plot=plot, label='mc', fig_name=figname+'_ini')
    index_max_ini = np.argmax(peaks_y_ini)
    variables.max_peak = peaks_ini[index_max_ini]
    variables.peak = peaks_ini
    mrp = (peaks_ini[index_max_ini] / (peak_widths_p_ini[index_max_ini][2] - peak_widths_p_ini[index_max_ini][1]))
    print('Mass resolving power for the highest peak (MRP --> m/m_2-m_1):', mrp)
    for i in range(len(peaks_ini)):
        print('Peaks ', i, 'is at location and height: ({:.2f}, {:.2f})'.format(peaks_ini[i], peaks_y_ini[i]), 'peak window sides (half-maximum) are: ({:.2f}, {:.2f})'.format(peak_widths_p_ini[i][1], peak_widths_p_ini[i][2]))

interact_manual(mc_hist_plot, mc=fixed(mc), bin_size=widgets.FloatText(value=0.1)
               ,prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), lim_mc=widgets.IntText(value=150)
               ,percent=widgets.IntText(value=50), selector=fixed('rect'), plot=fixed(True), figname=fixed(figname));


interactive(children=(FloatText(value=0.1, description='bin_size'), IntText(value=50, description='prominence'…

In [8]:
element_selected = wd.density_field_selection()
display(element_selected)

Dropdown(description='Element', options=((1, 'H', 52.7, 0.0), (2, 'He', 28.7, 0.0), (3, 'Li', 46.3, 14.0), (4,…

In [55]:
avg_dens = element_selected.value[2]
field_evap = element_selected.value[3]
flight_path_length = flightPathLength_d.value 

In [56]:
# mc_low = range_data['mc_low'].tolist()
# mc_up = range_data['mc_up'].tolist()
# df_s = data.copy(deep = True)
# mask = np.logical_and((mc_low[0] < mc), (mc < mc_up[0])).squeeze()
# print(mask[mask==True].shape)
# df_s = df_s[(df_s['mc_c (Da)'] > mc_low[0]) & (df_s['mc_c (Da)'] < mc_up[0])]
# df_s.reset_index(inplace=True, drop=True)
# df_s.shape

In [57]:
# reconstruction_calculation_plot(dld_x, dld_y, dld_highVoltage, data, range_data, element_percentage, flight_path_length=110, kf=4, det_eff=0.7, icf=1.4, field_evap=19, avg_dens=60.2, rotary_fig_save=False, range_file_exist=True)

In [58]:
button_range = widgets.Button(
    description='load range',
)

@button_range.on_click
def open_file_on_click(b):
    global range_path
    range_path = dataset_path_qt.gui_fname().decode('ASCII')
button_range


Button(description='load range', style=ButtonStyle())

In [59]:
range_data = data_tools.read_hdf5_through_pandas(range_path)
range_data

,element,isotope,charge,mc_low,mc_up,color
0,H,1,1,1.100636,1.300752,#b2aa2d
1,Al,27,2,13.607863,13.908036,#e7e0d1
2,Al,27,1,26.815494,27.315783,#e7e0d1


In [60]:
if range_data is not None:
    element_percentage = [0.01] * len(range_data['element'].tolist())
    element_percentage = str(element_percentage)
else:
    element_percentage = str([0.01])

interact_manual(reconstruction.reconstruction_calculation_plot, dld_x=fixed(dld_x), dld_y=fixed(dld_y), dld_highVoltage=fixed(dld_highVoltage), flight_path_length=fixed(flight_path_length),
                kf=widgets.FloatText(value=4), det_eff=widgets.FloatText(value=0.7), icf=widgets.FloatText(value=1.4),
                field_evap=widgets.FloatText(value=field_evap),
               avg_dens=widgets.FloatText(value=avg_dens), element_percentage=widgets.Textarea(value=element_percentage),
               rotary_fig_save=widgets.Dropdown(options=[('True', True), ('False', False)], value=False), data=fixed(data), range_data=fixed(range_data),
                range_file_exist=widgets.Dropdown(options=[('True', True), ('False', False)], value=True), figname=figname);

C:\Users\mehrp\AppData\Local\Temp\ipykernel_13572\873979118.py:7: DeprecationWarning:

on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').



interactive(children=(Textarea(value='[0.01, 0.01, 0.01]', description='element_percentage'), FloatText(value=…

In [61]:
data

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse (V),start_counter,t (ns),t_c (nm),x_det (cm),y_det (cm),pulse_pi,ion_pp,tof_c (ns)
0,0.0,0.0,0.0,27.352441,27.401002,5269.834473,5269.834473,54436.0,635.421132,0.0,-1.595020,-1.661878,0,1,0.0
1,0.0,0.0,0.0,27.228966,27.522766,5269.834473,5269.834473,54903.0,635.901192,0.0,0.646286,2.129878,467,1,0.0
2,0.0,0.0,0.0,27.086795,27.582728,5269.834473,5269.834473,55353.0,648.101574,0.0,-0.175102,3.155020,450,1,0.0
3,0.0,0.0,0.0,27.282802,26.913047,5269.834473,5269.834473,55831.0,636.010920,0.0,1.964327,-1.999347,478,1,0.0
4,0.0,0.0,0.0,27.132434,27.273478,5269.834473,5269.834473,56022.0,633.350016,0.0,1.381714,1.757388,191,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898626,0.0,0.0,0.0,248.242374,248.363850,5711.471680,5711.471680,37587.0,1716.831720,0.0,0.369306,-1.330776,83,1,0.0
1898627,0.0,0.0,0.0,27.040206,27.320343,5711.471680,5711.471680,38162.0,600.993972,0.0,0.423429,0.630367,575,1,0.0
1898628,0.0,0.0,0.0,27.665819,27.881308,5711.471680,5711.471680,38200.0,606.096324,0.0,0.560327,0.200571,38,1,0.0
1898629,0.0,0.0,0.0,27.019084,27.072135,5711.471680,5711.471680,38467.0,600.026994,0.0,0.926449,-0.633551,267,1,0.0


In [62]:
def projection(dld_x, dld_y, dld_highVoltage, data, range_data, element_percentage, figname, x_or_y):
    ax = plt.figure().add_subplot(111)
    
    phases = range_data['element'].tolist()
    colors = range_data['color'].tolist()
    mc_low = range_data['mc_low'].tolist()
    mc_up = range_data['mc_up'].tolist()
    charge = range_data['charge'].tolist()
    isotope = range_data['isotope'].tolist()
    mc = data['mc_c (Da)']
        
    
    element_percentage = element_percentage.replace('[', '')
    element_percentage = element_percentage.replace(']', '')
    element_percentage = element_percentage.split(',')
    
    for index, elemen in enumerate(phases):
        df_s = data.copy(deep = True)
        df_s = df_s[(df_s['mc_c (Da)'] > mc_low[index]) & (df_s['mc_c (Da)'] < mc_up[index])]
        df_s.reset_index(inplace=True, drop=True)
        remove_n = int(len(df_s) - (len(df_s) * float(element_percentage[index])))
#         print(len(df_s))
        drop_indices = np.random.choice(df_s.index, remove_n, replace=False)
        df_subset = df_s.drop(drop_indices)
#         print(len(df_subset))
        if x_or_y == 'x':
            ax.scatter(df_subset['x (nm)'], df_subset['z (nm)'], s=2, label=elemen)
        elif x_or_y == 'y':
            ax.scatter(df_subset['y (nm)'], df_subset['z (nm)'], s=2, label=elemen)

    data_loadcrop.rectangle_box_selector(ax)
    plt.connect('key_press_event', selectors_data.toggle_selector)
    ax.xaxis.tick_top()
    ax.invert_yaxis()
    if x_or_y == 'x':
            ax.set_xlabel('X (nm)')
    elif x_or_y == 'y':
            ax.set_xlabel('Y (nm)')
    ax.xaxis.set_label_position('top')
    ax.set_ylabel('Z (nm)')
    plt.legend()
    plt.savefig(variables.result_path + '\\output_{fn}.png'.format(fn=figname))
    plt.show()

In [63]:
interact_manual(projection, dld_x=fixed(dld_x), dld_y=fixed(dld_y), dld_highVoltage=fixed(dld_highVoltage), data=fixed(data), range_data=fixed(range_data), element_percentage=widgets.Textarea(value=element_percentage)
               ,figname=fixed(figname), x_or_y=widgets.Dropdown(options=['x', 'y'], value='x') );

interactive(children=(Textarea(value='[0.01, 0.01, 0.01]', description='element_percentage'), Dropdown(descrip…

In [65]:
print('Min x (nm):', variables.selected_x1, 'Max x (nm):', variables.selected_x2)
print('Min y (nm):', variables.selected_x1, 'Max y (nm):', variables.selected_x2)
print('Min z (nm):', variables.selected_y1, 'Max z (nm):', variables.selected_y2)

Min x (nm): -12.396549393212052 Max x (nm): 5.353214132807274
Min y (nm): -12.396549393212052 Max y (nm): 5.353214132807274
Min z (nm): 1.9137814718131363 Max z (nm): 7.989791557400423


In [28]:
# save the new data
name_save_file = variables.result_path + '//' + dataset_name + '.h5'
data_tools.store_df_to_hdf(name_save_file, data, 'df')

In [29]:
# save data as epos file
epos = hdf_to_leap.hdf5_to_epos(data, path=variables.result_path, name=dataset_name + '.epos')
pos = hdf_to_leap.hdf5_to_pos(data, path=variables.result_path, name=dataset_name + '.POS')

In [ ]:
name_save_file = variables.result_path + '//' + dataset_name + '.csv'
data.to_csv(name_save_file, index = False)
